In [72]:
import sqlite3
import pandas as pd

In [73]:
class GroupByDB:
    
    def __init__(self, dbpath):
        self.dbpath = dbpath
        self.conn = sqlite3.connect(dbpath)
        self.cursor = self.conn.cursor()
    
    def __del__(self):
        self.conn.close()
    
    def total_sales_by_year(self):
        query = """
        SELECT
            SUBSTRING(InvoiceDate,1,4) as Year,
	        SUM(Total) YearlySales
        FROM
	        invoices i
        GROUP BY
	        Year
        """
        return pd.read_sql_query(query, self.conn)
    
    def total_sales_by_country(self):
        query = """
        SELECT
	        Country,
	        SUM(Total) SalesByCountry
        FROM
	        invoices i
        JOIN customers c 
        ON i.CustomerId = c.CustomerId 
        GROUP BY
	        Country
        """
        return pd.read_sql_query(query, self.conn)
    
    def track_count(self):
        query = """
        SELECT
	        PlaylistId,
	        COUNT(TrackId) Tracks_in_playlist 
        FROM
	        playlist_track pt 
        GROUP BY
	        PlaylistId
        """
        return pd.read_sql_query(query, self.conn)
    
    def track_length(self):
        query = """
        SELECT
	        pt.PlaylistId,
	        COUNT(pt.TrackId) TracksInPlaylist,
	        SUM(t.Milliseconds)/(1000*60) PlaylistLengthMinutes
        FROM
	        playlist_track pt 
        JOIN tracks t 
        ON pt.TrackId = t.TrackId 
        GROUP BY
	        pt.PlaylistId
        """
        return pd.read_sql_query(query, self.conn)
    
    def playlist_names(self):
        query = """
        SELECT
	        pt.PlaylistId,
	        p.Name,
	        COUNT(pt.TrackId) TracksInPlaylist,
	        SUM(t.Milliseconds)/(1000*60) PlaylistLengthMinutes
        FROM
	        playlist_track pt 
        JOIN tracks t 
        ON pt.TrackId = t.TrackId 
        JOIN playlists p  
        ON pt.PlaylistId = p.PlaylistId 
        GROUP BY
	        pt.PlaylistId
        """
        return pd.read_sql_query(query, self.conn)

In [74]:
db = GroupByDB("chinook.db")

1. get total sales for all years using invoice table
- you will want to use SUBSTR to get the year from the invoice date
- you will want to use SUM to get the total sales for each year

In [75]:
db.total_sales_by_year()

,Year,YearlySales
0,2009,449.46
1,2010,481.45
2,2011,469.58
3,2012,477.53
4,2013,450.58


2. get total sales for each country - use invoice table
- you will also need to join with the customer table - those have country info

In [76]:
db.total_sales_by_country()

,Country,SalesByCountry
0,Argentina,37.62
1,Australia,37.62
2,Austria,42.62
3,Belgium,37.62
4,Brazil,190.10
5,Canada,303.96
6,Chile,46.62
7,Czech Republic,90.24
8,Denmark,37.62
9,Finland,41.62


3. a  Count tracks in each playlist - use playlist_track table

In [77]:
db.track_count()

,PlaylistId,Tracks_in_playlist
0,1,3290
1,3,213
2,5,1477
3,8,3290
4,9,1
5,10,213
6,11,39
7,12,75
8,13,25
9,14,25


3. b extra challenge get total track lenght in minutes for each playlist
- you will need to join with the track table

In [78]:
db.track_length()

,PlaylistId,TracksInPlaylist,PlaylistLengthMinutes
0,1,3290,14628
1,3,213,8351
2,5,1477,6645
3,8,3290,14628
4,9,1,4
5,10,213,8351
6,11,39,158
7,12,75,362
8,13,25,112
9,14,25,126


3. c cherry on top - provide names of these playlists
- so you will want to join with the playlist table as well

In [79]:
db.playlist_names()

,PlaylistId,Name,TracksInPlaylist,PlaylistLengthMinutes
0,1,Music,3290,14628
1,3,TV Shows,213,8351
2,5,90’s Music,1477,6645
3,8,Music,3290,14628
4,9,Music Videos,1,4
5,10,TV Shows,213,8351
6,11,Brazilian Music,39,158
7,12,Classical,75,362
8,13,Classical 101 - Deep Cuts,25,112
9,14,Classical 101 - Next Steps,25,126


In [80]:
del db